## setup

In [ ]:
from unsloth.chat_templates import standardize_sharegpt, get_chat_template, train_on_responses_only
import unsloth # ABOVE PYTORCH (cuz dependency things)
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig, DataCollatorForSeq2Seq
from huggingface_hub import login
from dotenv import load_dotenv
from datasets import load_dataset
import os
import wandb
from peft import LoraConfig, get_peft_model
from pipeline.main import run_eval
import json
from IPython.display import display, HTML
from trl import SFTConfig, SFTTrainer
import torch as t

In [ ]:
load_dotenv()
login(token = os.environ['HF_TOKEN'])
wandb.login()

## FT Dataset

### Unsloth 3/n

In [ ]:
max_seq_length = 32768 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

In [ ]:
with open("/root/srf-project/data/codenet_questions/filtered_problem_descriptions.json", "r") as f:
    problem_descriptions = json.load(f)

def get_filtered_dataset(lang):
    dataset = load_dataset('iNeil77/CodeNet', lang, split='train')
    dataset = dataset.select_columns(['p_id', 'language', 'status', 'code'])
    dataset = dataset.filter(lambda x: x['status']=='Accepted')
    dataset = dataset.filter(lambda x: x['p_id'] in problem_descriptions.keys())

    return dataset

def get_train_test(dataset, train_size=10000, test_size=500):
    shuffled = dataset.shuffle(seed=47)
    train_set = shuffled.select(range(train_size))
    test_set = shuffled.select(range(train_size, train_size + test_size))

    return train_set, test_set

def add_description(row):
    description = problem_descriptions[row['p_id']]
    row['description'] = description
    return row

def add_conversations(row):
    preface = 'Read the following problem description. Fully implement a solution in Java. Your response should only contain the code, no explanations.\n\n'
    problem_description = row['description']
    code = row['code']

    row['conversations'] = [
        {
            'role': 'user', 
            'content': preface + problem_description,
        },
        {
            'role': 'assistant', 
            'content': code,
        }
    ]

    return row

def chat_format(row):
    convos = row['conversations']
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { 'text' : texts}

In [ ]:
my_dataset = get_filtered_dataset('Java')
train_set, test_set = get_train_test(my_dataset)

train_set = train_set.map(add_description, remove_columns=['p_id', 'language', 'status'])
test_set = test_set.map(add_description, remove_columns=['p_id', 'language', 'status'])

train_set = train_set.map(add_conversations, remove_columns=['code', 'description'])
test_set = test_set.map(add_conversations, remove_columns=['code', 'description'])

train_set = train_set.map(chat_format, batched=True)
test_set = test_set.map(chat_format, batched=True)

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    # auto_find_batch_size = True,
    gradient_accumulation_steps = 4,
    warmup_ratio = 0.05,
    num_train_epochs = 1,
    eval_strategy = 'steps',
    eval_steps = 0.05,
    learning_rate = 2e-4,
    bf16 = True,
    logging_steps = 5,
    logging_first_step=True,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    save_steps = 0.2,
    save_total_limit = 3,
    output_dir = "outputs",
    report_to = "wandb", 
    # run_name = '___'
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_set,
    eval_dataset = test_set,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 8,
    args = training_args,
    # packing = False, 
)

In [ ]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
wandb.finish()

### Unsloth Experimentation!

In [ ]:
max_seq_length = 32768 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
)

In [ ]:
with open("/root/srf-project/data/codenet_questions/filtered_problem_descriptions.json", "r") as f:
    problem_descriptions = json.load(f)

def get_filtered_dataset(lang):
    dataset = load_dataset('iNeil77/CodeNet', lang, split='train')
    dataset = dataset.select_columns(['p_id', 'language', 'status', 'code'])
    dataset = dataset.filter(lambda x: x['status']=='Accepted')
    dataset = dataset.filter(lambda x: x['p_id'] in problem_descriptions.keys())

    return dataset

def get_train_test(dataset, train_size=10000, test_size=500):
    shuffled = dataset.shuffle(seed=47)
    train_set = shuffled.select(range(train_size))
    test_set = shuffled.select(range(train_size, train_size + test_size))

    return train_set, test_set

def add_description(row):
    description = problem_descriptions[row['p_id']]
    row['description'] = description
    return row

def add_final_prompt_new(row):
    preface = 'Read the following problem description. Fully implement a solution in Java. Your response should only contain the code, no explanations.\n\n'
    problem_description = row['description']
    code = row['code']

    conversational_prompt_completion = {
        'prompt': [{
            'role': 'user',
            'content': preface + problem_description
        }],
        'completion': [{
            'role': 'assistant',
            'content': code
        }],
    }

    return conversational_prompt_completion

In [ ]:
filtered = get_filtered_dataset('Java')
train_set, test_set = get_train_test(filtered)

train_set = train_set.map(add_description)
test_set = test_set.map(add_description)

final_train_set = train_set.map(add_final_prompt_new, remove_columns=['description', 'code', 'p_id', 'status', 'language'])
final_test_set = test_set.map(add_final_prompt_new, remove_columns=['description', 'code', 'p_id', 'status', 'language'])

In [ ]:
sft_args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
        completion_only_loss=True,
        dataset_kwargs={'skip_prepare_dataset': True}
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_train_set,
    # max_seq_length = max_seq_length,
    # packing = False, # Can make training 5x faster for short sequences.
    args = sft_args,
)

In [ ]:
trainer_stats = trainer.train()

### Load Model / Dataset

In [ ]:
'''
model_name = 'meta-llama/Llama-3.1-8B-Instruct'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=t.float16,
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, pad_side="left")
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    quantization_config=bnb_config,
)
tokenizer.pad_token = tokenizer.eos_token'''

# model_name = 'unsloth/Meta-Llama-3.1-8B-Instruct'
model_name = 'unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit'


tokenizer = AutoTokenizer.from_pretrained(model_name, pad_side="left")
tokenizer.pad_token = tokenizer.eos_token


model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    torch_dtype="auto",
)

In [ ]:
with open("/root/srf-project/data/codenet_questions/filtered_problem_descriptions.json", "r") as f:
    problem_descriptions = json.load(f)

def get_filtered_dataset(lang):
    dataset = load_dataset('iNeil77/CodeNet', lang, split='train')
    dataset = dataset.select_columns(['p_id', 'language', 'status', 'code'])
    dataset = dataset.filter(lambda x: x['status']=='Accepted')
    dataset = dataset.filter(lambda x: x['p_id'] in problem_descriptions.keys())

    return dataset

def get_train_test(dataset, train_size=10000, test_size=500):
    shuffled = dataset.shuffle(seed=47)
    train_set = shuffled.select(range(train_size))
    test_set = shuffled.select(range(train_size, train_size + test_size))

    return train_set, test_set

filtered = get_filtered_dataset('Java')
train_set, test_set = get_train_test(filtered, test_size=8)

In [ ]:
def add_description(row):
    description = problem_descriptions[row['p_id']]
    row['description'] = description
    return row

train_set = train_set.map(add_description)
test_set = test_set.map(add_description)

In [ ]:
def add_final_prompt(row):
    description = row['description'].strip()
    code = row['code']

    final_prompt = '\n'.join([description, '<answer>', code, '</answer>'])
    row['final_prompt'] = final_prompt

    return row

train_set = train_set.map(add_final_prompt)
test_set = test_set.map(add_final_prompt)

In [ ]:
def tokenize(record):
    final_prompt = record['final_prompt']
    msg = [
        {'role': 'user', 'content': final_prompt}
    ]

    tokens = tokenizer.apply_chat_template(msg, add_generation_prompt=True, return_dict=True)

    return tokens

final_train_set = train_set.map(tokenize, num_proc=32)
final_test_set = test_set.map(tokenize, num_proc=32)

final_train_set = final_train_set.select_columns(['input_ids', 'attention_mask'])
final_test_set = final_test_set.select_columns(['input_ids', 'attention_mask'])

### FineTuning

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=0.00,
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, lora_config)

In [ ]:
def get_save_dir():
    idx = 0
    base_path = '/root/srf-project/playground/llama3-java-finetune-'
    already_existing = True

    while already_existing:
        idx += 1
        path = base_path + str(idx)
        
        if not os.path.exists(path):
            already_existing = False
            os.mkdir(path)
            return path
        
output_path = get_save_dir()

training_args = TrainingArguments(
    output_dir=output_path,
    # eval_strategy='steps',
    # eval_steps=0.1,
    # eval_on_start=True,
    per_device_train_batch_size=1, 
    # auto_find_batch_size=True,
    gradient_accumulation_steps=1,
    # gradient_checkpointing=True,
    torch_empty_cache_steps=1,
    dataloader_num_workers=16,
    dataloader_persistent_workers=True,
    learning_rate=5e-4,
    num_train_epochs=1,  
    bf16=True,
    save_steps=0.2,
    save_total_limit=3,
    logging_steps=0.02,
    report_to="wandb",
    logging_first_step=True,
    run_name='chat_template',
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # For causal LM
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=final_train_set,
    eval_dataset=final_test_set,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
wandb.finish()

### RunEval

In [ ]:
args = {
    'model': 'hf/local',
    'model_path': '/root/srf-project/playground/llama3-java-finetune-1',
    'device': 'auto',
    'torch_dtype': 'auto'
}

run_eval('java', args, samples=164)

### GPU Deets

In [ ]:
import torch as t
import gc

free_memory, total_memory = t.cuda.mem_get_info()

# Convert bytes to GB
free_memory_gb = free_memory / (1024 * 1024 * 1024)
total_memory_gb = total_memory / (1024 * 1024 * 1024)
mem_used = t.cuda.device_memory_used() / (1024 ** 3)

print(f"Free GPU Memory: {free_memory_gb:.2f} GB")
print(f"Total GPU Memory: {total_memory_gb:.2f} GB")
print(f'Memory Used: {mem_used:.2f} GB')

In [ ]:
print(t.cuda.memory_allocated() / 1024**2, "MB allocated")
print(t.cuda.memory_reserved() / 1024**2, "MB reserved")

In [ ]:
print(t.cuda.memory_allocated() / 1024**2, "MB allocated")
print(t.cuda.memory_reserved() / 1024**2, "MB reserved")